In [33]:
import pandas as pd
import numpy as np
import openpyxl as op

### Class function with analytics and utility methods

In [2]:
class Analytics:

    def __init__(
            self,
            data_file_name, 
            template_file_name,
            data_file_sheet = 2): 
        
        self.data_file_name = data_file_name
        self.template_file_name = template_file_name 
        self.data_file_sheet = data_file_sheet
        self.wb = None

    ## Add path argument import sys 
    def load_data(self): 
        port = pd.read_excel(self.data_file_name, sheet_name=self.data_file_sheet)
        return port 

    ## Add path argument import sys 
    def load_template(self): 
        wb = op.load_workbook(self.template_file_name)
        self.wb = wb
        return wb
    
    def mvwa(self, df, fieldname): 
        """
        the goal of function, market value 
        arg: data type 
        df: dataframe of a portfolio, following data file scheme 
        category: string of data field name
        return market value weighted data field 
        """

        df['Weight'] = df['Market_Value']/df['Market_Value'].sum()
        weighted_analytics = df['Weight'] * df[category]
        return weighted_analytics.sum()
    
    def get_mv_percent(self, portolio, fieldname):
        """
        port: dataframe of a portfolio, following data file scheme 
        category: string of data field name to be aggregated on 
        return market value percentage of a category 
        """

        mv_percent = port.groupby([category]).agg({'Market_Value': np.sum})/port.Market_Value.sum()
        mv_percent = mv_percent.reset_index()
        mv_percent[category] = mv_percent[category].astype(str)
        
        return mv_percent
    

    def get_template_val(self, st_row, ed_row, col, category): 
        """
        st_row: start row number of template 
        ed_row: end row number of template 
        col: col to read in 
        category: string of data field name to be reported on 
        """
        vals = []
        ws = self.wb.active 
        for r in ws.iter_rows(min_row = st_row, min_col = col, max_row = ed_row, max_col = col):
            for cell in r:
                vals.extend([cell.value])
        df = pd.DataFrame({category: vals})
        df[category]=df[category].astype(str)
        
        return df 
    
    def populate_data(self, df_template, df_results, category):
        """
        df_template: df from get_template_val 
        df_results: df from get_mv_percent 
        category: string of data field name to be reported on
        """ 

        df = df_template.merge(df_results, on=category, how="left")
        df = df.fillna(0)

        return df
    
    def write_results(self, st_row, ed_row, col, results, template_file_name): 
        """
        template_results: dataframe or list of results to be written out 
        """
        ws = self.wb.active 
        for r in ws.iter_rows(min_row = st_row, min_col = col, max_row = ed_row, max_col = col): 
            for cell in r: 
                if isinstance(results, list): 
                    cell.value = results[cell.row - st_row]
                else: 
                    cell.value = results.Market_Value[cell.row - st_row]
        
        template_file_name = "template_file_name"
        self.wb.save(template_file_name)


### Initialize with respective data and template files

In [35]:
file_name = 'Case_Study_Data_AIS.xlsx'
template_name = 'AnalyticsTemplate.xlsx'

case_study = Analytics(file_name, template_name)
wb = case_study.load_template()

In [36]:
port_totl = case_study.load_data()
port_totl.head()

,Portfolio,Primary_Asset_ID,Asset_Type,Coupon_Type,Rating,Industry,Market_Value,OAS,Yield,Duration
0,Ares,CINAVYTI,Loan,Floating,BB,Media,703181,317.0,4.17,0.115
1,Ares,DUHVUIVB,Bond,Fixed,BB-,Services,955601,387.0,5.87,1.023
2,Ares,BKJJDAID,Bond,Fixed,CCC,Healthcare,732398,519.0,6.19,6.176
3,Ares,JKSOSIKI,Bond,Fixed,BB-,Consumer Goods,1885559,588.0,6.88,6.295
4,Ares,HUIYGUHC,Loan,Floating,B,Financial Services,956697,485.0,6.85,0.236


### Calculate respective summary metrics and mv percentages

#### 1. Summary metrics calculations on port segments 

In [37]:
port_loan = port_totl[port_totl['Asset_Type'] == 'Loan'].copy()
port_bond = port_totl[port_totl['Asset_Type'] == 'Bond'].copy()
port_fixed = port_totl[port_totl['Coupon_Type'] == 'Fixed'].copy()
port_float = port_totl[port_totl['Coupon_Type'] == 'Floating'].copy()

In [38]:
port_list = [port_totl, port_bond, port_loan]
analytics_list = ['Yield', 'OAS']

port_list2 = [port_totl, port_fixed, port_float]
analytics_list2 = ['Duration']

In [39]:
num_asset = port_totl.Primary_Asset_ID.value_counts().values.sum()

result_summary =[num_asset]

for a in analytics_list: 
    for p in port_list:
        result = case_study.mvwa(p, a)
        result_summary.extend([result])

for a in analytics_list2: 
    for p in port_list2:
        result = case_study.mvwa(p, a)
        result_summary.extend([result])
        
market_value = port_totl.Market_Value.sum()
result_summary.extend([market_value])
len(result_summary)

11

#### 2. mv percentage by rating

In [40]:
df_rating = case_study.get_mv_percent(port_totl, 'Rating')
df_rating.head()

,Rating,Market_Value
0,B,0.201154
1,B+,0.143007
2,B-,0.072982
3,BB,0.049749
4,BB+,0.136047


In [41]:
result_rating = case_study.get_template_val(st_row=3, ed_row = 25, col=5, category='Rating')
result_rating = case_study.populate_data(result_rating, df_rating, 'Rating')
result_rating.head()

,Rating,Market_Value
0,AAA,0.0
1,AA+,0.0
2,AA,0.0
3,AA-,0.0
4,A+,0.0


#### 3. mv percentage by industry

In [42]:
df_industry = case_study.get_mv_percent(port_totl, 'Industry')
df_industry.head()

,Industry,Market_Value
0,Automotive,0.091238
1,Basic Industry,0.109781
2,Capital Goods,0.064788
3,Consumer Goods,0.027810
4,Energy,0.048622


In [43]:
result_industry = case_study.get_template_val(st_row=32, ed_row = 49, col=5, category='Industry')
result_industry = case_study.populate_data(result_industry, df_industry, 'Industry')
result_industry.head()

,Industry,Market_Value
0,Automotive,0.091238
1,Banking,0.000000
2,Basic Industry,0.109781
3,Capital Goods,0.064788
4,Consumer Goods,0.027810


### Write out results to template 

In [44]:
case_study.write_results(st_row=3, ed_row = 25, col=6, results = result_rating)
case_study.write_results(st_row=32, ed_row = 49, col=6, results = result_industry)
case_study.write_results(st_row=3, ed_row = 13, col=3, results = result_summary)